## State Student Spending Case Study
### Examining any relationship between student sucess or lack thereof with spending


In [31]:
#import necessry modules

import pandas as pd

In [101]:
#import data
Grad = pd.read_csv('Data/Completion1318edit.csv')
Drop = pd.read_csv('Data/Dropout1318edit.csv')
StateSpend = pd.read_excel('Data/StateSpendingStudents.xlsx')
Pop = pd.read_csv('Data/Pop10-19.csv')

In [79]:
#Check Grad Data. Rename columns so they match years in other files.
oldname = Grad.columns
newname = ['State', 2013, 2014, 2015, 2016, 2017, 2018 ]
Grad = Grad.rename(columns=dict(zip(oldname, newname)))
Grad.head()

,State,2013,2014,2015,2016,2017,2018
0,United States,81.4,82.3,83.2,84.1,84.6,85.3
1,Alabama,80.0,86.3,89.3,87.1,89.3,90.0
2,Alaska,71.8,71.1,75.6,76.1,78.2,78.5
3,Arizona,75.1,75.7,77.4,79.5,78.0,78.7
4,Arkansas,84.9,86.9,84.9,87.0,88.0,89.2


In [80]:
Drop.head()

,State,2013,2014,2015,2016,2017,2018
0,United States,6.8,6.3,6.0,5.8,5.4,5.3
1,Alabama,8.9,7.7,7.4,6.9,6.6,5.4
2,Alaska,6.2,2.8,8.8,5.6,5.3,4.4
3,Arizona,9.4,8.5,9.2,7.6,8.0,7.7
4,Arkansas,6.8,6.8,6.7,7.2,5.7,5.6


In [98]:
#Limit to desired years
#StateSpend = StateSpend.loc[:,'2013':'2016']
#rowname = StateSpend.iloc[:,0]
#print(rowname)

0                  Alabama
1                   Alaska
2                  Arizona
3                 Arkansas
4               California
5                 Colorado
6              Connecticut
7                 Delaware
8     District of Columbia
9                  Florida
10                 Georgia
11                  Hawaii
12                   Idaho
13                Illinois
14                 Indiana
15                    Iowa
16                  Kansas
17                Kentucky
18               Louisiana
19                   Maine
20                Maryland
21           Massachusetts
22                Michigan
23               Minnesota
24             Mississippi
25                Missouri
26                 Montana
27                Nebraska
28                  Nevada
29           New Hampshire
30              New Jersey
31              New Mexico
32                New York
33          North Carolina
34            North Dakota
35                    Ohio
36                Oklahoma
3

In [102]:
Pop.head()

,Geographic Area,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,"308,745,538","308,758,105","309,321,666","311,556,874","313,830,990","315,993,715","318,301,008","320,635,163","322,941,311","324,985,539","326,687,501","328,239,523"
1,Northeast,"55,317,240","55,318,443","55,380,134","55,604,223","55,775,216","55,901,806","56,006,011","56,034,684","56,042,330","56,059,240","56,046,620","55,982,803"
2,Midwest,"66,927,001","66,929,725","66,974,416","67,157,800","67,336,743","67,560,379","67,745,167","67,860,583","67,987,540","68,126,781","68,236,628","68,329,004"
3,South,"114,555,744","114,563,030","114,866,680","116,006,522","117,241,208","118,364,400","119,624,037","120,997,341","122,351,760","123,542,189","124,569,433","125,580,448"
4,West,"71,945,553","71,946,907","72,100,436","72,788,329","73,477,823","74,167,130","74,925,793","75,742,555","76,559,681","77,257,329","77,834,820","78,347,268"
